In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="gemma2-9b-it", temperature=0)

In [ ]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template(
    template="""You are a chat assistant called {name} and your task is to say random unreleated stuff""",
    input_variables=["name"]
)

human_prompt = HumanMessagePromptTemplate.from_template(
    template="""
The user wants you to tell something related to the following topic:
---
    {topic}
---
Don't really care about what he wants, generate some random stuff. Do not include anything else expect the output.
""",
    input_variables=["topic"]
)

In [10]:
print(human_prompt.format(topic="LangChain").content)


The user wants you to tell something related to the following topic:
---
    LangChain
---
Don't really care about what he wants, generate some random stuff. Do not include anything else expect the output.



In [11]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate([system_prompt, human_prompt])
print(chat_prompt.format(name="Random", topic="LangChain"))

System: You are a chat assistant called Random and your task is to say random unreleated stuff
Human: 
The user wants you to tell something related to the following topic:
---
    LangChain
---
Don't really care about what he wants, generate some random stuff. Do not include anything else expect the output.



In [12]:
chain = (
    {"name": lambda x: x["name"], "topic": lambda x: x["topic"]} |
    chat_prompt |
    llm
)

In [ ]:
print(chain.invoke({"name": "Random", "topic": "LangChain"}).content)

Purple elephants are surprisingly good at hide and seek. 



In [18]:
from pydantic import BaseModel, Field

class OutPutStructure(BaseModel):
    name: str = Field(None, description="Name of the chat assistant")
    topic: str = Field(None, description="The actual user topic")
    output: str = Field(None, description="The output generated but the chat assistant")

structured_llm = llm.with_structured_output(OutPutStructure)

In [19]:
new_chain = (
    {"name": lambda x: x["name"], "topic": lambda x: x["topic"]} |
    chat_prompt |
    structured_llm
)

In [20]:
new_chain.invoke({"name": "Random", "topic": "LangChain"})

OutPutStructure(name='Random', topic='Random Stuff', output='The quick brown fox jumps over the lazy dog.')